In [14]:
from arcgis.gis import GIS
from data_engineering.utils import fetch_charging_stations, fetch_traffic_accidents
from urban_traffic.utils import fetch_traffic_data, read_traffic_features
import os
import pandas as pd

### Additional dependencies
We load the necessary variables using `dotenv`.  
Please, ensure that you setup before: `pip install python-dotenv`  
Furthermore, put a `.env` file into the root of this repository.

In [2]:
from dotenv import load_dotenv

def load_vars():
    load_dotenv(dotenv_path="../../../../.env")
    return os.getenv("ARCGIS_API_KEY"), os.getenv("TRAFFIC_DATA_FILE"), os.getenv("TRAFFIC_FEATURES")

In [3]:
api_key, traffic_data_filepath, traffic_features_filepath = load_vars()

### Connect to your GIS portal

In [4]:
gis = GIS(api_key=api_key)

### Use a map for the area of interest

In [78]:
def create_map(gis: GIS, location: str = "Ludwig-Erhard-Anlage 1, 60327 Frankfurt am Main, Germany"):
    map_view = gis.map(location)
    map_view.basemap.basemap = "osm"
    map_view.zoom = 17
    return map_view

In [79]:
map_view = create_map(gis)
map_view

Map(center=[6465801.0751674855, 962955.6795531422], extent={'xmin': 962882.1493646716, 'ymin': 6465580.1136933…

### Spatial filter using the area of interest

In [129]:
distance_in_meters = 150
lon, lat = map_view.center[:2]
traffic_features = read_traffic_features(traffic_features_filepath, lon=lon, lat=lat, meters=distance_in_meters)
traffic_features[["trip", "person", "vehicle_type"]]

,trip,person,vehicle_type
0,9775,6094,Car
1,9775,6094,Car
2,9775,6094,Car
3,12069,7524,Bike
4,12069,7524,Bike
...,...,...,...
5444,3228,2002,Bike
5445,3228,2002,Bike
5446,3228,2002,Bike
5447,1806,1120,Car


In [161]:
traffic_features.dtypes

OBJECTID           int64
SHAPE           geometry
trip               int64
person             int64
vehicle_type      object
trip_time         object
dtype: object

In [162]:
import pandas as pd

def prepare_traffic(traffic_df: pd.DataFrame) -> pd.DataFrame:
    # Extract hours, minutes, and seconds
    traffic_df["trip_time"] = pd.to_datetime(traffic_df["trip_time"], format="%Y-%m-%dT%H:%M:%S")
    traffic_df["hour"] = traffic_df["trip_time"].dt.strftime("%H").astype(int)
    traffic_df["minute"] = traffic_df["trip_time"].dt.strftime("%M").astype(int)
    traffic_df["second"] = traffic_df["trip_time"].dt.strftime("%S").astype(int)
    traffic_df = traffic_df.drop(columns=["trip_time"], axis=1)

    # Fill empty values for vehicle_type
    traffic_df = traffic_df.fillna({"vehicle_type": "pedestrian"})

    # Convert all vehicle types to lower case
    traffic_df["vehicle_type"] = traffic_df["vehicle_type"].str.lower()

    # Unique vehicle types
    vehicle_types = traffic_df["vehicle_type"].unique()

    # Create binary columns
    for vehicle_type in vehicle_types:
        traffic_df[vehicle_type] = traffic_df["vehicle_type"].apply(lambda vt: 1 if vt == vehicle_type else 0).astype(int)
    traffic_df = traffic_df.drop(columns=["vehicle_type"], axis=1)
    
    return traffic_df

In [163]:
prepared_traffic_features = prepare_traffic(traffic_features.copy())
prepared_traffic_features

,OBJECTID,SHAPE,trip,person,hour,minute,second,car,bike,pedestrian
0,24964,"{""x"": 8.65026980600004, ""y"": 50.11366486900005...",9775,6094,1,5,9,1,0,0
1,24975,"{""x"": 8.650076004000027, ""y"": 50.1136734090000...",9775,6094,1,5,10,1,0,0
2,24986,"{""x"": 8.649882559000048, ""y"": 50.1136853030000...",9775,6094,1,5,11,1,0,0
3,229379,"{""x"": 8.650534005000054, ""y"": 50.1136312120000...",12069,7524,3,50,59,0,1,0
4,229404,"{""x"": 8.650538966000056, ""y"": 50.1135955960000...",12069,7524,3,51,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...
5444,16266910,"{""x"": 8.651493778000031, ""y"": 50.1132211080000...",3228,2002,19,52,48,0,1,0
5445,16267081,"{""x"": 8.65155403600005, ""y"": 50.11321575200003...",3228,2002,19,52,49,0,1,0
5446,16267255,"{""x"": 8.651614294000069, ""y"": 50.1132103950000...",3228,2002,19,52,50,0,1,0
5447,16321768,"{""x"": 8.65022818600005, ""y"": 50.11369300700005...",1806,1120,19,58,19,1,0,0


In [158]:
prepared_traffic_features.dtypes

OBJECTID         int64
SHAPE         geometry
trip             int64
person           int64
hour             int32
minute           int32
second           int32
car              int32
bike             int32
pedestrian       int32
dtype: object

### Visualize commute traffic

In [159]:
def filter_commute_cars(traffic_df: pd.DataFrame) -> pd.DataFrame:
    return traffic_df.query("car == 1 and 7 < hour and hour < 10")

In [160]:
commute_cars = filter_commute_cars(prepared_traffic_features.copy())
commute_cars

,OBJECTID,SHAPE,trip,person,hour,minute,second,car,bike,pedestrian
736,2286311,"{""x"": 8.650260810000077, ""y"": 50.1136643500000...",7015,4368,8,0,21,1,0,0
737,2286921,"{""x"": 8.650066991000074, ""y"": 50.1136738510000...",7015,4368,8,0,22,1,0,0
738,2287515,"{""x"": 8.649873591000073, ""y"": 50.1136860210000...",7015,4368,8,0,23,1,0,0
739,2352185,"{""x"": 8.65024780400006, ""y"": 50.11366498700005...",4614,2864,8,2,10,1,0,0
740,2352791,"{""x"": 8.650053986000046, ""y"": 50.1136744880000...",4614,2864,8,2,11,1,0,0
...,...,...,...,...,...,...,...,...,...,...
3079,7119801,"{""x"": 8.64895797500003, ""y"": 50.11347199200003...",631,402,9,59,55,1,0,0
3080,7120530,"{""x"": 8.649109127000031, ""y"": 50.1134819290000...",631,402,9,59,56,1,0,0
3081,7121227,"{""x"": 8.649301682000043, ""y"": 50.1134648500000...",631,402,9,59,57,1,0,0
3082,7121912,"{""x"": 8.649494237000056, ""y"": 50.1134477720000...",631,402,9,59,58,1,0,0


In [143]:
commute_map_view = create_map(gis)
commute_cars.spatial.plot(commute_map_view)
commute_map_view.zoom_to_layer(commute_cars)
commute_map_view

Map(center=[6465801.0751674855, 962955.6795531422], extent={'xmin': 962752.5686014448, 'ymin': 6465537.9578022…